# <h1 align="center"><font color="red">Previsão de série temporal com Prophet em Python</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

# <font color="gree">Contextualizando</font>

<font color="orange">A previsão de série temporal pode ser desafiadora, pois há muitos métodos diferentes que você pode usar e muitos hiperparâmetros diferentes para cada método.

A `biblioteca prophet` é uma biblioteca de código aberto projetada para fazer previsões (`forecasts`) para conjuntos de dados de `séries temporais univariadas`. É fácil de usar e projetado para encontrar automaticamente um bom conjunto de hiperparâmetros para o modelo, em um esforço para fazer previsões hábeis para dados com tendências e estrutura sazonal por padrão.</font>



Instalação:
```
$ pip install prophet
```

In [4]:
# verifique a versão de prophet:
import prophet
# Versão:
print('Prophet %s' % prophet.__version__)

Prophet 1.1.5


# <font color="gree">Dataset</font>

<font color="orange">Usaremos o conjunto de dados mensais de vendas de carros.

É um conjunto de dados de `série temporal univariado` padrão que contém uma `tendência` e uma `sazonalidade`. O conjunto de dados tem 108 meses de dados e uma previsão de persistência ingénua (naive) pode atingir um `erro médio absoluto` de cerca de `3.235` vendas, proporcionando um limite de erro mais baixo.</font>

# <font color="gree">Carregar e resumir conjunto de dados</font>

<font color="orange">Primeiro, vamos carregar e resumir o conjunto de dados.

O Profeta exige que os dados estejam nos DataFrames do Pandas. Portanto, carregaremos e resumiremos os dados usando Pandas.

Podemos carregar os dados diretamente do URL chamando a função read_csv() Pandas e, em seguida, resumir a forma (número de linhas e colunas) dos dados e visualizar as primeiras linhas de dados.</font>